In [1]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']

#  this doc would be on monthly & customer level
#  however the opportunity might not applicable for this.
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n"

In [260]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,df_cost,df_esti,rel, df_bcaw]:
    colname_modi(i)
 
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

In [261]:
def data_prepare():
    """
    data prepare opportunity .
    """
    df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

opportunity = data_prepare();opportunity.head(3)
# del contract_summary

,account_name_en,contract_end_date,account_id,bg,contract_term,milestone,opportunity_record,product1,product2,sector,annual_average_gross_profit_base,annual_average_gross_profit,annual_average_revenue_base,annual_average_revenue,contract_value,contract_end_yr
0,ABC,2021-06-28,e0209d33-e3aa-ea11-a812-000d3a851174,TSC,12.0,Closed-Canceled,New Business,Warehousing,Transportation,Technology,5000.0,5000.0,50000.0,50000.0,50000.0,2021
1,ADIDAS,2018-09-29,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,36.0,Contract Signed,New Business,Warehousing,Warehousing,Retail,950.4,1188.0,10777.6,13472.0,40416.0,2018
2,ADIDAS,2019-10-30,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,12.0,Contract Signed,Straightforward Renewals,Warehousing,Transportation,Retail,1087.2,1359.0,13123.2,16404.0,0.0,2019


In [17]:
df_list = [opportunity,df_bill,df_cost,df_esti]


In [185]:
def check_ouname(df):
    print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

for i in df_list:
    check_ouname(i)

del df_list,i

Empty DataFrame
Columns: [0]
Index: [] 	
              0
13  cost_center 	
         0
1  ou_code 	
              0
12  cost_center 	


In [262]:
"""
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4)
    df = df.groupby(['account_name_en','account_name_cn', substr_year]).agg({
        **kwargs
    })
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

---
# billing

In [263]:
"""
~billing~
bill detail table now having 50+ bms_types. which is unacceptalble/
"""
df_bill['month'] = df_bill['bms_bill_end_date'].astype(str).str.slice(0,6) 
df_bill = df_bill.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
bms = df_bill.pivot_table(index= ['cost_center', 'month'],\
    columns= 'category',
    values='bms_amount',).reset_index().fillna(0)

In [277]:
bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
    cost_center = set, 
    人力 = 'sum',
    场地 = 'sum',
    增值 = 'sum',
    报关代理 = 'sum',
    操作 = 'sum',
    短驳运输 = 'sum',
    索赔及返利 = 'sum',
    设备使用 = 'sum' ).reset_index()

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/2509375719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


In [281]:
bms_cus_yr.head(2)

,account_name_en,account_name_cn,month,cost_center,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用
0,ASML,阿斯麦,2020,"{ASMLXNJLHS, ASMLXXAEPS, ASMLXSHZCS, ASMLXBJKC...",1.327222e+06,2.650783e+06,277290.273019,8.301644e+05,351038.906930,3.930000e+04,0.0,218274.787736
1,ASML,阿斯麦,2021,"{ASMLXTJXQS, ASMLXNJLHS, ASMLXSHZCS, ASMLXXAEP...",2.027038e+06,3.116845e+06,614587.514906,2.113024e+06,180360.231132,1.676826e+06,0.0,274845.181649


---
# cost

In [213]:
# def tst(substr_year,**kwargs):
#     df_bill[substr_year] = df_bill[substr_year].astype(str).str.slice(0,4)
#     return  df_bill.groupby([substr_year, 'cost_center']).agg({
#         **kwargs
#     }).reset_index()

# tst(substr_year = 'bms_bill_end_date' ,\
#     bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

In [212]:
# test = df_bill.head(39)
# test.pivot_table(index= ['bms_bill_id', 'cost_center'],\
#     columns= ['bms_fee_type'],
#     values='bms_amount',)

In [282]:
"""
# cost
"""
df_cost = df_cost.dropna(axis=1, how = 'all').fillna(0)
frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)
df_cost = df_cost.loc[:, frame[frame[0] == False].index.tolist()]
del frame
df_cost.head()

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/1996595441.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)


,id,ou_code,level,month,rev,gp,hr_cost,staff_cost,staff_cost_dir,hour_dir,...,staff_cost_bizos,staff_cost_bizos_actual,staff_roi_nmom,staff_cost_cm_mom,staff_cost_cm_nmom,rev_target,staff_cost_target,gp_target,hc_target,src_inc_day
0,1907,ABBOTBJXWS,site,201912,150.0,-66.0,53745.36,59.0,59.0,1144.0,...,357.72,357.72,4.85,3.77,-6.52,0.00,0.00,0.00,0.0,20211111
1,10047,ABBOTBJXWS,site,202102,119.0,-45.0,52033.54,54.0,0.0,0.0,...,514.18,514.18,-3.38,-6.43,-0.90,141.53,53.33,-44.43,0.0,20211111
2,6079,ABBOTBJXWS,site,202006,115.0,-53.0,45693.69,38.0,0.0,0.0,...,810.23,810.23,-12.92,0.32,3.13,0.00,0.00,0.00,0.0,20211111
3,1107,ABBOTBJXWS,site,201907,100.0,-140.0,45630.13,48.0,48.0,0.0,...,369.16,369.16,-4.56,0.25,-4.45,0.00,0.00,0.00,0.0,20211111
4,10560,ABBOTBJXWS,site,202103,65.0,-75.0,51766.79,54.0,0.0,0.0,...,514.18,514.18,-13.10,-0.52,-4.98,159.96,53.27,-25.95,0.0,20211111


In [283]:
df_cost[['hr_cost', 
'hc',
'hc_os',
'total_working_hrs',
'staff_cost_fixed',
'staff_cost_vari',
'staff_cost_os_fixed',
'staff_cost_os_vari',
'staff_cost_bizos_actual',]]

,hr_cost,hc,hc_os,total_working_hrs,staff_cost_fixed,staff_cost_vari,staff_cost_os_fixed,staff_cost_os_vari,staff_cost_bizos_actual
0,53745.36,4.23,0.0,1144.0,39483.01,13904.63,0.0,0.0,357.72
1,52033.54,4.25,0.0,0.0,40337.87,11181.49,0.0,0.0,514.18
2,45693.69,4.25,0.0,0.0,33485.63,11397.83,0.0,0.0,810.23
3,45630.13,4.21,0.0,0.0,34592.80,10668.17,0.0,0.0,369.16
4,51766.79,4.25,0.0,0.0,39397.14,11855.47,0.0,0.0,514.18
...,...,...,...,...,...,...,...,...,...
2535,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00
2536,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00
2537,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00
2538,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00


In [285]:
cost_cus_yr = to_cust_level(df_cost, 'month',\
     rev = 'sum',
     gp = 'sum',
     hr_cost  = 'sum', 
     hc  = 'sum',
     hc_os  = 'sum',
     total_working_hrs  = 'sum',
     staff_cost_fixed  = 'sum',
     staff_cost_vari  = 'sum',
     staff_cost_os_fixed  = 'sum',
     staff_cost_os_vari  = 'sum',
     staff_cost_bizos_actual  = 'sum'
     ).reset_index()

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19972/2509375719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


In [293]:
test = bms_cus_yr.merge(cost_cus_yr, on = ['account_name_en', 'account_name_cn','month'], how = 'inner')

SyntaxError: invalid syntax (Temp/ipykernel_19972/2332363091.py, line 1)

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

